In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

顶点训练：分布式超参数调整

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/hyperparameter_tuning/distributed-hyperparameter-tuning.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/hyperparameter_tuning/distributed-hyperparameter-tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/hyperparameter_tuning/distributed-hyperparameter-tuning.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

##概述

这本笔记本演示了如何使用Vertex Training运行超参数调整作业，以发现ML模型的最佳超参数值。为了加快训练过程，使用了`tf.distribute`模块中的`MirroredStrategy`来在单台机器的多个GPU上分发训练。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets)中的[马或人数据集](https://www.tensorflow.org/datasets/catalog/horses_or_humans)。训练的模型预测一张图像是马还是人。

### 目标

在本笔记本中，您将在Docker容器中的Python脚本中创建一个自定义训练的模型。您将学习如何修改用于超参数调整的训练应用程序代码，并使用Python SDK提交一个Vertex Training超参数调整作业。

执行的步骤包括：

* 创建一个用于训练模型的Vertex AI自定义作业。
* 使用Python SDK启动超参数调整作业。
* 清理资源。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解有关[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)的信息，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成一个成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Google Cloud笔记本**，您的环境已经满足运行此笔记本所需的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合该笔记本的要求。您需要以下内容：

• Google Cloud SDK
• Git
• Python 3
• virtualenv
• 在使用 Python 3 的虚拟环境中运行的 Jupyter 笔记本

Google Cloud 的《配置 Python 开发环境指南》和 Jupyter 的安装指南提供了满足这些要求的详细说明。以下步骤提供了一组简化的说明：

1. [安装并初始化 Cloud SDK。](https://cloud.google.com/sdk/docs/)
2. [安装 Python 3。](https://cloud.google.com/python/setup#installing_python)
3. [安装 virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用 Python 3 的虚拟环境。激活虚拟环境。
4. 要安装 Jupyter，请在命令行终端中运行 `pip3 install jupyter`。
5. 要启动 Jupyter，请在命令行终端中运行 `jupyter notebook`。
6. 在 Jupyter Notebook 仪表板中打开此笔记本。

### 安装其他软件包

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform

重新启动内核

在安装额外的软件包之后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 选择 GPU 运行时

**确保在 GPU 运行时下运行此笔记本，如果您有这个选项的话。在 Colab 中，选择“运行时 --> 更改运行时类型 > GPU”**

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额度，用于支付计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. 如果您在本地运行此笔记本，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保
Cloud SDK将正确的项目用于本笔记本中的所有命令。

**注意**：Jupyter会将以`!`为前缀的行作为Shell命令运行，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目 ID

**如果您不知道您的项目 ID**，您可以使用 `gcloud` 获取您的项目 ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = ""  # @param {type:"string"}

设置项目ID

In [ ]:
! gcloud config set project $PROJECT_ID

时间戳

如果您在一个在线教程会话中，您可能会使用一个共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您需要为每个实例会话创建一个时间戳，并将其附加到您在本教程中创建的资源名称之后。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐户

**如果您正在使用谷歌云笔记本**，您的环境已经被验证过了。可以跳过这一步。

**如果您正在使用Colab**，请运行下面的单元格，并按提示进行身份验证OAuth。

**否则**，请按照以下步骤操作：

1. 在云控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

4. 在**将此服务帐号授予项目访问权限**部分，点击**Role**下拉列表。在过滤框中输入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中输入"存储对象管理员"，并选择**存储对象管理员**。

5. 点击*创建*。包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中，将您的服务帐号密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

当您使用云 SDK 提交自定义训练作业时，您将需要提供一个临时存储桶。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

您还可以更改 `REGION` 变量，该变量用于本笔记本的其余操作。请确保选择一个支持 Vertex AI 服务的[区域](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。您不能使用多区域存储桶来训练 Vertex AI。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

In [ ]:
print(BUCKET_URI)

只有当您的存储桶尚未存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查内容来验证对您的云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

###导入库并定义常量

In [ ]:
import os
import sys

from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

### 编写Dockerfile

将您的代码容器化的第一步是创建一个Dockerfile。在Dockerfile中，您将包含运行映像所需的所有命令，例如安装必要的库和设置用于训练代码的入口点。

此Dockerfile使用深度学习容器 TensorFlow Enterprise 2.5 GPU Docker 镜像。Google Cloud 上的深度学习容器预装了许多常见的 ML 和数据科学框架。下载完该镜像后，这个Dockerfile会安装 [CloudML Hypertune](https://github.com/GoogleCloudPlatform/cloudml-hypertune) 库，并设置用于训练代码的入口点。

In [ ]:
%%writefile Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5
WORKDIR /

# Installs hypertune library
RUN pip install cloudml-hypertune

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

### 创建培训应用程序代码

接下来，您可以创建一个trainer目录，其中包含一个名为`task.py`的脚本，其中包含您的培训应用程序的代码。

In [ ]:
# Create trainer directory

! mkdir trainer

在下一个单元格中，您需要编写训练脚本 `task.py` 的内容。该文件从 TensorFlow 数据集下载 _horses or humans_ 数据集，并使用 `tf.keras` 功能模型在 `tf.distribute` 模块中使用 `MirroredStrategy` 进行训练。

以下是一些特定于使用超参数调整服务的组件：

* 脚本导入了 `hypertune` 库。请注意，Dockerfile 包含了安装 hypertune 库的指令。
* 函数 `get_args()` 为您想要调整的每个超参数定义了一个命令行参数。在本示例中，将调整的超参数是学习率、优化器中的动量值以及模型中最后一个隐藏层中的单元数。传递给这些参数的值然后用于设置代码中相应的超参数。
* 在 `main()` 函数的结尾处，使用 hypertune 库来定义要优化的度量标准。在本示例中，将优化的度量标准是验证准确性。此度量标准被传递给一个 `HyperTune` 实例。

In [ ]:
%%writefile trainer/task.py

import argparse
import hypertune
import tensorflow as tf
import tensorflow_datasets as tfds

def get_args():
  """Parses args. Must include all hyperparameters you want to tune."""

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate', required=True, type=float, help='learning rate')
  parser.add_argument(
      '--momentum', required=True, type=float, help='SGD momentum value')
  parser.add_argument(
      '--units',
      required=True,
      type=int,
      help='number of units in last hidden layer')
  parser.add_argument(
      '--epochs',
      required=False,
      type=int,
      default=10,
      help='number of training epochs')
  args = parser.parse_args()
  return args


def preprocess_data(image, label):
  """Resizes and scales images."""

  image = tf.image.resize(image, (150, 150))
  return tf.cast(image, tf.float32) / 255., label


def create_dataset(batch_size):
  """Loads Horses Or Humans dataset and preprocesses data."""

  data, info = tfds.load(
      name='horses_or_humans', as_supervised=True, with_info=True)

  # Create train dataset
  train_data = data['train'].map(preprocess_data)
  train_data = train_data.shuffle(1000)
  train_data = train_data.batch(batch_size)

  # Create validation dataset
  validation_data = data['test'].map(preprocess_data)
  validation_data = validation_data.batch(64)

  return train_data, validation_data


def create_model(units, learning_rate, momentum):
  """Defines and compiles model."""

  inputs = tf.keras.Input(shape=(150, 150, 3))
  x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(units, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(inputs, outputs)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.SGD(
          learning_rate=learning_rate, momentum=momentum),
      metrics=['accuracy'])
  return model


def main():
  args = get_args()

  # Create Strategy
  strategy = tf.distribute.MirroredStrategy()

  # Scale batch size
  GLOBAL_BATCH_SIZE = 64 * strategy.num_replicas_in_sync  
  train_data, validation_data = create_dataset(GLOBAL_BATCH_SIZE)

  # Wrap model variables within scope
  with strategy.scope():
    model = create_model(args.units, args.learning_rate, args.momentum)

  # Train model
  history = model.fit(
      train_data, epochs=args.epochs, validation_data=validation_data)

  # Define Metric
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=args.epochs)


if __name__ == '__main__':
  main()

### 构建容器

在接下来的单元格中，您将构建容器并将其推送到Google容器注册表。

In [ ]:
# Set the IMAGE_URI
IMAGE_URI = f"gcr.io/{PROJECT_ID}/horse-human:hypertune"

In [ ]:
# Build the docker image
! docker build -f Dockerfile -t $IMAGE_URI ./

In [ ]:
# Push it to Google Container Registry:
! docker push $IMAGE_URI

### 在Vertex AI上创建并运行超参数调整作业

一旦您的容器被推送到Google容器注册表，您可以使用Vertex SDK来创建并运行超参数调整作业。

您需要定义以下规格：
* `worker_pool_specs`： 指定机器类型和Docker镜像的字典。此示例定义了一个具有一个`n1-standard-4`机器和两个`NVIDIA_TESLA_T4` GPU的单节点集群。
* `parameter_spec`：指定要优化的参数的字典。字典键是分配给您的训练应用程序代码中每个超参数命令行参数的字符串，字典值是参数规范。参数规范包括超参数的类型、最小/最大值和缩放比例。
* `metric_spec`：指定要优化的度量标准的字典。字典键是您在训练应用程序代码中设置的`hyperparameter_metric_tag`，值是优化目标。

In [ ]:
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 2,
        },
        "replica_count": 1,
        "container_spec": {"image_uri": IMAGE_URI},
    }
]

metric_spec = {"accuracy": "maximize"}

parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "units": hpt.DiscreteParameterSpec(values=[64, 128, 512], scale=None),
}

创建一个 `CustomJob`。

In [ ]:
print(BUCKET_URI)

In [ ]:
# Create a CustomJob

JOB_NAME = "horses-humans-hyperparam-job" + TIMESTAMP

my_custom_job = aiplatform.CustomJob(
    display_name=JOB_NAME,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=BUCKET_URI,
)

然后，创建和运行`HyperparameterTuningJob`。

需要注意一些参数:

* `max_trial_count`: 设置服务运行试验的最大上限。建议的做法是从较小数量的试验开始，先了解您选择的超参数对结果的影响，然后再逐步增加。

* `parallel_trial_count`: 如果使用并行试验，服务会分配多个训练处理集群。创建任务时指定的worker pool spec会用于每个单独的训练集群。增加并行试验的数量会减少超参数调整作业运行的时间，但可能会降低作业的整体有效性。这是因为默认调整策略使用先前试验的结果来指导随后试验中的数值分配。

* `search_algorithm`: 可用的搜索算法为grid, random或default（None）。默认选项会应用贝叶斯优化来搜索可能的超参数数值空间，是推荐的算法。

In [ ]:
# Create and run HyperparameterTuningJob

hp_job = aiplatform.HyperparameterTuningJob(
    display_name=JOB_NAME,
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=15,
    parallel_trial_count=3,
    project=PROJECT_ID,
    search_algorithm=None,
)

hp_job.run()

点击输出中生成的链接可以在云控制台上查看您的运行情况。当作业完成时，您将看到调优试验的结果。

将以下英文文本翻译为中文：![console_ui_results]（调优结果.png）

清理

要清理本项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI